In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [0]:
#SILENCING THE FALSE POSITIVE WARNINGS
import warnings
warnings.simplefilter('always')
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

In [0]:
#IMPORTING DEPENDENCIES
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D, Activation

In [0]:
#IMPORTING DATASET
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv', index_col=False)
df = data

In [0]:
#Filling Missing Values in Data
#print(df.isna().sum())
df[['title', 'author']] = df[['title', 'author']].fillna(value = 'Missing Value')
df = df.dropna()
df['length'] = df.iloc[:,3].str.len()
#print(df.isna().sum())
df[df['length'] < 50].count()
df = df.drop(df['text'][df['length'] < 50].index, axis=0)
df_reverse = pd.DataFrame()


#Categorical to Numeric
for col_name in df.columns:
    if(df[col_name].dtype == 'object'):
        df[col_name]= df[col_name].astype('category')
        d = dict(enumerate(df[col_name].cat.categories))
        df[col_name] = df[col_name].cat.codes
        df_reverse[col_name+"_code"] = df[col_name]
        df_reverse[col_name] = df[col_name].map(d)

features_cols = ['id', 'title', 'author', 'text']
#FEATURES AND LABELS
X = df[features_cols]
Y = df.label

In [0]:
#PREPARING TRAINING DATASET AND TEST DATASET
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [106]:
#DECISION TREE CLASSIFIER - UNPRUNDED TREE
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
dtc_unpruned = DecisionTreeClassifier()
dtc_unpruned.fit(X_train, Y_train)
Y_pred_dtc_unpruned = dtc_unpruned.predict(X_test)
acc_score_dtc_unpruned = round(metrics.accuracy_score(Y_test,Y_pred_dtc_unpruned) * 100)
print("Accuracy of DecisionTreeClassifier:", acc_score_dtc_unpruned, "%")

Accuracy of DecisionTreeClassifier: 88.0 %


In [0]:
X_test_cp = X_test
#Decoding Data - ONE TIME STEP - DO NOT REPEAT
df_reverse.set_index('title_code', inplace=False)
title_dict = df_reverse.to_dict()['title']
df_reverse.set_index('author_code', inplace=False)
author_dict = df_reverse.to_dict()['author']
df_reverse.set_index('text_code', inplace=False)
text_dict = df_reverse.to_dict()['text']

X_test_cp['title'] = X_test_cp['title'].map(title_dict)
X_test_cp['author'] = X_test_cp['author'].map(author_dict)
X_test_cp['text'] = X_test_cp['text'].map(text_dict)
X_test_cp.set_index('id', inplace=True)

In [0]:
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp['Prediction'] = Y_pred_dtc_unpruned
X_test_cp['Prediction'].replace([0,1],['Fake News','Relaible News'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [113]:
#PREDICTION RESULTS
X_test_cp.tail()

,title,author,text,Prediction
id,,,,
12652,Commissioner Starts to Press Cleveland Indians...,The New York Times,"At the Golden Globes afterparties, hoi polloi ...",Fake News
13401,Cosmology/ Love & the Afterlife,noreply@blogger.com (Alexander Light),Trending Articles: Trending Articles: ‘Chairma...,Fake News
5882,Fordham Basketball Coach Recruits With an Acad...,Dr. Patrick Slattery,Hollywood’s biggest stars took to social media...,Fake News
11500,"Carrie Fisher, Russia, Shinzo Abe: Your Wednes...",TFPP Writer,This post was originally published on this sit...,Fake News
14754,Ken Blackwell: Media ‘Borking’ Sessions Becaus...,Mac Slavo,"IQUITOS, Peru — Venezuela took its stronges...",Relaible News


In [0]:
#Visualizing DecisionTree
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(dtc_unpruned, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = features_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('fakeNews.png')
Image(graph.create_png())